# Random forest importance

In [1]:
%store -r X_256
%store -r y_256
%store -r X_test_256
%store -r y_test_256
%store -r feature_names_256

In [2]:
# Import thư viện
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [3]:
import matplotlib.pyplot as plt

# Reshape data
# Data ban đầu X(13166, 20, 128), y(13166)

# Reshape X về dạng 2d => X_2d(263320, 128), giữ nguyên 128 và đó là các đặc trưng
X_2d = X_256.reshape(-1, X_256.shape[-1])
# Reshape y thành y_reshaped(263320) bằng cách lặp mỗi phần tử trong y 20 lần
y_reshaped = np.repeat(y_256, 20)

In [4]:
# Khởi tạo mô hình Random Forest
model = RandomForestClassifier(n_estimators=500)
# Nó chỉ định số lượng cây quyết định sẽ được xây dựng và kết hợp để tạo thành khu rừng ngẫu nhiên trong thuật toán random forest.

In [ ]:
# Huấn luyện mô hình
model.fit(X_2d, y_reshaped)

In [ ]:
# Lấy tầm quan trọng của đặc trưng
feature_importances = model.feature_importances_

# In tầm quan trọng của đặc trưng
print(feature_importances)

In [ ]:
feature_importances.shape

In [ ]:
feature_names_256

In [ ]:
# Tạo dataframe gồm tên đặc trưng và tầm quan trọng
feature_importances_df = pd.DataFrame({"feature_name": feature_names_256, "importance": feature_importances})

# Sắp xếp dataframe theo tầm quan trọng
feature_importances_df = feature_importances_df.sort_values("importance", ascending=False)

# Lấy index của dataframe
indices = feature_importances_df.index

# In index
print(indices)

In [ ]:
# Xác định số đặc trưng sẽ giữ lại để train  model
k = 128
top_k_indices = indices[:k]

In [ ]:
# Giữ lại k đặc trưng tốt nhất trong tệp dữ liệu ban đầu
selected_X = X_256[:,:,top_k_indices]
selected_X_test = X_test_256[:,:,top_k_indices]

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Khởi tạo mô hình
model = keras.Sequential()

model.add(layers.Input(shape = (20, k, 1)))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5, 5), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 32 bộ lọc kích thước 5x5 và hàm kích hoạt ReLU
model.add(layers.Conv2D(32, (5,5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Convolutional Layer với 64 bộ lọc kích thước 3x3 và hàm kích hoạt ReLU
model.add(layers.Conv2D(64, (5, 5), activation='relu', padding='same'))

# Thêm lớp Max Pooling 2D để giảm kích thước đầu vào đi 2 lần
model.add(layers.MaxPooling2D((2, 2)))

# Thêm lớp Flatten để làm phẳng đầu ra của lớp trước khi đi vào các lớp fully connected
model.add(layers.Flatten())

# Thêm lớp fully connected (Dense Layer) với 64 units và hàm kích hoạt ReLU
model.add(layers.Dense(64, activation='relu'))

# Thêm lớp Dropout để giảm hiện tượng overfitting
model.add(layers.Dropout(0.5))

# Thêm lớp fully connected (Dense Layer) cuối cùng với số lớp đầu ra phụ thuộc vào bài toán của bạn
model.add(layers.Dense(5, activation='softmax'))

# In thông tin mô hình
model.summary()

# Biên dịch mô hình
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(selected_X, y_256, epochs=10, batch_size=16)

In [ ]:
# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model.evaluate(selected_X_test, y_test_256)
print("Độ chính xác trên tập kiểm tra:", test_accuracy)

# Dự đoán các nhãn trên tập kiểm tra
y_pred = model.predict(selected_X_test)

# Chuyển đổi dự đoán thành nhãn dự đoán (lớp dự đoán)
y_pred_classes = np.argmax(y_pred, axis=1)

# Hiển thị ma trận nhầm lẫn
from sklearn.metrics import confusion_matrix, classification_report
confusion = confusion_matrix(y_test_256, y_pred_classes)
print("Ma trận nhầm lẫn:")
print(confusion)

# Hiển thị báo cáo phân loại
class_report = classification_report(y_test_256, y_pred_classes)
print("Báo cáo phân loại:")
print(class_report)

In [ ]:
import numpy as np

def micro_metrics(y_true, y_pred):
  """
  Calculates micro and macro average precision, recall, and F1-score for multi-class classification.

  Args:
      y_true: List of true labels (length should match the number of data points in y_pred).
      y_pred: Predicted class probabilities for each data point (shape: (num_data, num_classes)).

  Returns:
      A dictionary containing micro and macro average precision, recall, and F1-score.
  """

  # Ensure y_true and y_pred have compatible shapes
  if len(y_true) != y_pred.shape[0]:
    raise ValueError("y_true and y_pred must have the same number of data points.")

  # Count true positives (TP), false positives (FP), false negatives (FN) per class
  # Assuming y_pred contains class probabilities
  y_pred_argmax = np.argmax(y_pred, axis=1)  # Get the predicted class index
  n_classes = y_pred.shape[1]  # Number of classes

  class_tp = np.zeros(n_classes)
  class_fp = np.zeros(n_classes)
  class_fn = np.zeros(n_classes)
  for y_t, y_p in zip(y_true, y_pred_argmax):
    class_tp[y_t] += 1
    if y_p != y_t:
      class_fp[y_p] += 1
      class_fn[y_t] += 1

  # Calculate micro and macro averages
  # Micro averages (already calculated in previous version)
  micro_tp = np.sum(class_tp)
  micro_fp = np.sum(class_fp)
  micro_fn = np.sum(class_fn)
  micro_precision = micro_tp / (micro_tp + micro_fp) if (micro_tp + micro_fp) > 0 else 0
  micro_recall = micro_tp / (micro_tp + micro_fn) if (micro_tp + micro_fn) > 0 else 0
  micro_f1 = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall) if (micro_precision + micro_recall) > 0 else 0

  # Macro averages
  macro_precision = np.mean([p / (p + fp) for p, fp in zip(class_tp, class_fp) if (p + fp) > 0])
  macro_recall = np.mean([r / (r + fn) for r, fn in zip(class_tp, class_fn) if (r + fn) > 0])
  macro_f1 = 2 * (macro_precision * macro_recall) / (macro_precision + macro_recall) if (macro_precision + macro_recall) > 0 else 0

  return {
      'micro_precision': micro_precision,
      'micro_recall': micro_recall,
      'micro_f1': micro_f1,
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'macro_f1': macro_f1
  }

In [ ]:
# Calculate micro metrics
micro_results = micro_metrics(y_test_256, y_pred)

print("Micro Precision:", micro_results['micro_precision'])
print("Micro Recall:", micro_results['micro_recall'])
print("Micro F1-score:", micro_results['micro_f1'])

print("Macro Precision:", micro_results['macro_precision'])
print("Macro Recall:", micro_results['macro_recall'])
print("Macro F1-score:", micro_results['macro_f1'])

In [ ]:
from tabulate import tabulate
# Prepare table data
table_data = [
  ["Micro Precision", micro_results['micro_precision']],
  ["Micro Recall", micro_results['micro_recall']],
  ["Micro F1-score", micro_results['micro_f1']],
  ["Macro Precision", micro_results['macro_precision']],
  ["Macro Recall", micro_results['macro_recall']],
  ["Macro F1-score", micro_results['macro_f1']],
]

# Print the table using tabulate
print(tabulate(table_data, headers=["Metric", "Score"]))